In [2]:
!pip install numpy
!pip install scikit-surprise

# Load dataset

In [27]:
load_df = pd.read_pickle("../data/raw/scraped/usersong")
load_df.shape[0]

499860

In [28]:
#Make sure all the count is number
load_df[["count"]] = load_df[["count"]].apply(pd.to_numeric)

In [29]:
#Changed the column name here to fit more data
cols = ['user', 'song', 'count']
load_df = pd.DataFrame(load_df, columns = cols)

In [30]:
#Take a look at the current data
load_df.head()

,user,song,count
0,raedrexlre,Rex Orange County-Corduroy Dreams,9
1,raedrexlre,Keshi-blue,8
2,raedrexlre,Zeph-Lucky,8
3,raedrexlre,Pink Sweat$-17,7
4,raedrexlre,Childish Gambino-Feels Like Summer,6


In [31]:
load_df.tail()

,user,song,count
499855,ben_council,"Tyler, The Creator-See You Again (feat. Kali U...",18
499856,ben_council,"A$AP Rocky-Everyday (feat. Rod Stewart, Miguel...",17
499857,ben_council,A$AP Rocky-Praise the Lord (Da Shine) (feat. S...,17
499858,ben_council,BROCKHAMPTON-SUGAR,17
499859,ben_council,Childish Gambino-IV. Sweatpants,17


In [17]:
#Create the surprise dataset
from surprise import Dataset
from surprise import Reader
reader = Reader()
data = Dataset.load_from_df(load_df[['user', 'song', 'count']], reader)

# Use cross-validation iterators

In [18]:
from surprise import SVD
from surprise.model_selection import cross_validate
svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    234.2314220.7892238.6519231.22427.5961  
MAE (testset)     96.1026 95.8821 96.3084 96.0977 0.1740  
Fit time          9.75    9.89    10.14   9.92    0.16    
Test time         1.61    1.40    1.38    1.46    0.10    


{'test_rmse': array([234.23140561, 220.78918777, 238.65194695]),
 'test_mae': array([96.10259273, 95.882133  , 96.30835434]),
 'fit_time': (9.746262073516846, 9.889868021011353, 10.137805700302124),
 'test_time': (1.6122910976409912, 1.3996946811676025, 1.3827002048492432)}

In [32]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV


param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

231.25424320243565
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}


In [24]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import KFold

# define a cross-validation iterator
kf = KFold(n_splits=3)

algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 249.4192
RMSE: 230.2709
RMSE: 212.9140


In [26]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=.25)
# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 225.5981


225.59810865753366

In [19]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [23]:
svd.predict(uid="raedrexlre", iid="Childish Gambino-IV. Sweatpants")

Prediction(uid='raedrexlre', iid='Childish Gambino-IV. Sweatpants', r_ui=None, est=5, details={'was_impossible': False})